# Start

In [29]:
# !source /fs/project/PAS1066/zhang_anaconda/anaconda3/bin/activate

In [30]:
import cudf 
import cupy as cp
import cuml
from cuml.svm import SVR
from cuml.model_selection import train_test_split
from cuml.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import time
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [31]:
df = cudf.read_csv('ricky_data.csv')

In [32]:
df.describe()

,Avg_Proton_Energy_(MeV),Focal_Distance_(um),Intensity_(W_cm2),Max_Proton_Energy_(MeV),Pulse_Duration_(fs),Spot_Size_(FWHM um),Target_Thickness (um),Total_Proton_Energy_(MeV),laserE_(J),ratio_proE_laserE_
count,195932.000000,195932.000000,1.959320e+05,195932.000000,195932.0,195932.0,195932.000000,1.959320e+05,195932.000000,195932.000000
mean,0.000366,0.000000,2.764737e+18,0.001144,40.0,2.0,7.700000,3.650781e+04,0.006282,0.000000
std,0.000761,17.606862,3.446224e+18,0.002414,0.0,0.0,4.214569,1.212380e+05,0.007830,0.000001
min,0.000001,-30.000000,1.000000e+17,0.000002,40.0,2.0,0.450000,2.473942e+00,0.000227,0.000000
25%,0.000028,-15.000000,3.162278e+17,0.000086,40.0,2.0,4.050000,3.470979e+02,0.000719,0.000000
50%,0.000103,0.000000,1.129463e+18,0.000316,40.0,2.0,7.700000,2.503727e+03,0.002566,0.000000
75%,0.000364,15.000000,3.981072e+18,0.001126,40.0,2.0,11.350000,1.895828e+04,0.009046,0.000000
max,0.013656,30.000000,1.258925e+19,0.046147,40.0,2.0,14.950000,3.156126e+06,0.028605,0.000013


In [33]:
df.head()

,Avg_Proton_Energy_(MeV),Focal_Distance_(um),Intensity_(W_cm2),Max_Proton_Energy_(MeV),Pulse_Duration_(fs),Spot_Size_(FWHM um),Target_Thickness (um),Total_Proton_Energy_(MeV),laserE_(J),ratio_proE_laserE_
0,0.000027,-30.0,1.000000e+17,0.000095,40.0,2.0,0.45,419.734708,0.000227,3.812325e-07
1,0.000018,-30.0,1.258925e+17,0.000058,40.0,2.0,0.45,672.007779,0.000286,4.279997e-07
2,0.000035,-30.0,1.584893e+17,0.000140,40.0,2.0,0.45,1368.311547,0.000360,4.805758e-07
3,0.000047,-30.0,1.995262e+17,0.000198,40.0,2.0,0.45,1321.107443,0.000453,5.397115e-07
4,0.000063,-30.0,2.511886e+17,0.000190,40.0,2.0,0.45,1700.042262,0.000571,6.062659e-07


In [34]:
df2 = df.copy()

# Creating the data splits

In [35]:
X = df2.iloc[0:len(df2), [1, 2, 4, 5, 6, 8]]
y = df2.iloc[0:len(df2), 3]

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.67)

# Standardizing X_train

In [37]:
stand_scaler = StandardScaler()

In [38]:
stand_scaler.fit(X_train)

StandardScaler()

In [39]:
X_train_norm= stand_scaler.transform(X_train)

In [40]:
X_test_norm = stand_scaler.transform(X_test)

# Log transform of y_train


In [41]:
y_train_log = cp.log(y_train)

# Creating the model

In [42]:
svr = SVR(
    kernel='poly',
    degree= 3,
    gamma='auto',
    C=15,
    epsilon=0.001,
    coef0= 1.2
)

In [43]:
start_time = time.time()

In [44]:
svr.fit(X_train_norm, y_train_log)

SVR()

# Exponentiating the predictions

In [45]:
y_predict_train = cp.asnumpy(cp.exp(svr.predict(X_train_norm)))
y_predict_test = cp.asnumpy(cp.exp(svr.predict(X_test_norm)))

In [46]:
end_time = time.time()

# Transfering cupy arrays and cudf to numpy arrays

In [47]:
myar = np.zeros(len(y_predict_test))

In [48]:
for i in range(len(myar)):
    myar[i] = y_predict_test[i]

In [49]:
y_predict_test = myar

In [50]:
myar2 = np.zeros(len(y_test))

In [51]:
y_test = y_test.to_numpy()

# Summary

In [52]:
print(f'Size of training set| {len(X_train)}\nSize of testing set | {len(X_test)}')

Size of training set| 131274
Size of testing set | 64658


In [53]:
print(f'MSE: {mean_squared_error(y_test, y_predict_test)}')

MSE: 6.127417020133465e-07


In [54]:
print(f'r^2 score :{r2_score(y_test, y_predict_test)}')

r^2 score :0.8935011255888113


In [55]:
total_time = end_time - start_time
print(f'Wall Clock\n\nHours  | {total_time // 60**2}\nMinutes| {total_time // 60}\nSeconds| {np.abs(total_time - 60 * (total_time // 60 ))}')

Wall Clock

Hours  | 0.0
Minutes| 3.0
Seconds| 15.292898893356323


# Testing the time needed for varying amounts of testing sizes

In [ ]:
xSize = np.linspace(start = .25, stop= .95, num= 50)
yTime = np.zeros(len(xSize))
for i in range(len(xSize)):
    print(f'{i}th job')
    X_trainTest, X_testTest, y_trainTest, y_testTest = train_test_split(X, y, train_size = xSize[i])
    # 
    stand_scalerTest = StandardScaler()
    #
    stand_scalerTest.fit(X_trainTest)
    #
    X_train_normTest = stand_scalerTest.transform(X_trainTest)
    #
    X_test_normTest = stand_scalerTest.transform(X_testTest)
    #
    y_train_logTest = cp.log(y_trainTest)
    #
    svrTest = SVR(
        kernel='poly',
        degree= 3,
        gamma='auto',
        C=15,
        epsilon=0.001,
        coef0= 1.2
    )
    #
    start_timeTest = time.time()
    #
    svrTest.fit(X_train_normTest, y_train_logTest)
    #
    y_predict_trainTest = cp.asnumpy(cp.exp(svr.predict(X_train_normTest)))
    y_predict_testTest = cp.asnumpy(cp.exp(svr.predict(X_test_normTest)))
    #
    end_timeTest = time.time()
    #
    total_timeTest = end_timeTest - start_timeTest
    #
    yTime[i] = total_timeTest

0th job
1th job
2th job
3th job
4th job
5th job
6th job
7th job
8th job
9th job
10th job
11th job
12th job
13th job
14th job
15th job
16th job
17th job
18th job
19th job
20th job
21th job
22th job
23th job
24th job
25th job
26th job
27th job
28th job
29th job
30th job
31th job


In [ ]:
xSize = xSize * 195932

In [ ]:
plt.scatter(xSize, yTime)

In [ ]:
dfTime = pd.DataFrame({'xSize':xSize, 'yTime':yTime})

In [ ]:
dfTime.to_csv('time_needed2.csv')